In [ ]:
!pip install pyensembl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 10.1 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4183 sha256=fd080ecc7a9602b60584773ab411b8988fc779cbcabfd69e6b9bb08b12e07f6c
  Stor

In [ ]:
!pyensembl install --release 86 --species homo_sapiens

2024-02-12 04:43:48,443 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=86, species='homo_sapiens')
2024-02-12 04:43:48,444 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl86/Homo_sapiens.GRCh38.86.gtf.gz from URL https://ftp.ensembl.org/pub/release-86/gtf/homo_sapiens/Homo_sapiens.GRCh38.86.gtf.gz
2024-02-12 04:43:48,444 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-86/gtf/homo_sapiens/Homo_sapiens.GRCh38.86.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl86/Homo_sapiens.GRCh38.86.gtf.gz
2024-02-12 04:44:45,583 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl86/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-86/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2024-02-12 04:44:45,584 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-86/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to

In [ ]:
from google.colab import files
import gzip
import pandas as pd
import os

import itertools
import numpy as np
import pandas as pd


In [ ]:
# Prompt the user to input the measure type
measure = input("Enter the measure (choose from local, global, or query): ")


Enter the measure (choose from local, global, or query): local


In [ ]:
tNames_input = input("Enter the values for tNames separated by commas: ")

# Split the input string based on commas and remove leading/trailing whitespace
tNames = [name.strip() for name in tNames_input.split(',')]

# Store the list of tissues in a dictionary with key 'Name'
tissues = {'Name': tNames, 'exp':[]}

print("tissues['Name']:", tissues['Name'])


Enter the values for tNames separated by commas: MSBB
tissues['Name']: ['MSBB']


In [ ]:
tNames

['MSBB']

In [ ]:
tissues

{'Name': ['MSBB'], 'exp': []}

In [ ]:
for filename in tNames:
    # Upload the .gz file
    uploaded_files = files.upload()

    # Extract the first (and only) key from the uploaded files dictionary
    file_name = next(iter(uploaded_files))

    # Get the full path of the uploaded file
    file_path = os.path.join(os.getcwd(), file_name)

    # Append the file path to the 'exp' list in tissues dictionary
    tissues['exp'].append(file_path)

Saving Kidney_Cortex.v8.normalized_expression.bed.gz to Kidney_Cortex.v8.normalized_expression.bed.gz


In [ ]:
tissues

{'Name': ['MSBB'],
 'exp': ['/content/Kidney_Cortex.v8.normalized_expression.bed.gz']}

In [ ]:
for tissue in tissues:
  print(tissues)

{'Name': ['MSBB'], 'exp': ['/content/Kidney_Cortex.v8.normalized_expression.bed.gz']}
{'Name': ['MSBB'], 'exp': ['/content/Kidney_Cortex.v8.normalized_expression.bed.gz']}


In [ ]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import statsmodels.api as sm
import os
import re
import sqlite3
import json
import gzip
import time
import asyncio
import threading
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
from pyensembl import EnsemblRelease
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LinearRegression

# Load the Ensembl database
ensembl_db = EnsemblRelease(86)

# Function to retrieve gene names from Ensembl IDs
def get_gene_names(ensembl_gene_ids):
    gene_mapping = []
    for ensembl_gene_id in ensembl_gene_ids:
        try:
            gene = ensembl_db.gene_by_id(ensembl_gene_id)
            if gene:
                gene_mapping.append({"GENEID": ensembl_gene_id, "SYMBOL": gene.gene_name})
        except Exception as e:
            pass
    return pd.DataFrame(gene_mapping)

def remove_extra_ensembl_ids(tissue_df, HGNC_gene_mapping):
    print(HGNC_gene_mapping)
    merged_df = tissue_df.merge(HGNC_gene_mapping, left_on="gene_id", right_on="GENEID", how="left")
    merged_df = merged_df[merged_df["SYMBOL"].notnull()]
    symbols = merged_df["SYMBOL"]
    # Drop the unnecessary columns
    merged_df.drop(columns=["GENEID", "SYMBOL"], inplace=True)
    return merged_df, symbols


# Function to add a column to a DataFrame
def add_column(tissue_incomplete, column_name):
    num_genes = tissue_incomplete.shape[0]
    random_vector = np.random.uniform(low=0.0001, high=0.00011, size=num_genes)
    df = pd.DataFrame(tissue_incomplete)
    df[column_name] = random_vector
    tissue_complete = df.rename(columns={"random_vector": column_name})
    return tissue_complete.to_numpy()

# Function to retrieve tissue data
def get_tissue(tissue):
    exp_files = tissues['exp']  # Get the list of file paths for all tissues
    tissue_exp_file = exp_files[tissues['Name'].index(tissue)]  # Find the file path for the specified tissue
    print(tissue_exp_file)
    tissue_df = pd.read_csv(tissue_exp_file, delimiter='\t')
    if "Ensembl ID" in tissue_df.columns:
        tissue_df["gene_id"] = tissue_df["Ensembl ID"]
    else:
        tissue_df["gene_id"] = tissue_df["gene_id"].str[:15]
    ensembl_gene_list = tissue_df["gene_id"]
    HGNC_gene_mapping = get_gene_names(ensembl_gene_list)
    tissue_df, headers = remove_extra_ensembl_ids(tissue_df, HGNC_gene_mapping)
    gene_tpm = tissue_df.iloc[:, 5:].T
    gene_tpm.columns = headers
    return gene_tpm

# Function to remove covariate effect
def rm_cov_effect(tissue_name, gene_tpm, cov_path):
    try:
        print("C")
        covariates = pd.read_csv(cov_path, sep="\t").set_index("ID").T
        print("CC")
    except FileNotFoundError:
        print("B")
        print("Covariates file not found")
        return None
    print("A")
    headers = gene_tpm.columns
    common_samples = gene_tpm.index.intersection(covariates.index)
    gene_tpm = gene_tpm.loc[common_samples].T
    covariates = covariates.loc[common_samples]

    residuals = []
    for i, row in tqdm(gene_tpm.iterrows(), total=len(gene_tpm), desc="Genes Processed: ", unit="Gene(s)"):
        model = sm.OLS(row, sm.add_constant(covariates))
        result = model.fit()
        residuals.append(result.resid)

    tissue = pd.DataFrame(residuals).T
    tissue.columns = headers
    return tissue

# Function to create correlation matrix
def create_corr_matrix(tissue_list):
    print(tissue_list)
    n = len(tissue_list)
    tissues, sample_ids, genes = [], [], []
    print("\n[Reading gene expression data for each tissue]")
    print("-------------------------------------------------")
    for i in tqdm(range(n), total=n, desc="Tissues Processed: ", unit=" Tissue(s)"):
        print(tissue_list[i])
        tissue = get_tissue(tissue_list[i])
        tissues.append(tissue)
        sample_ids.append(set(tissue.index))
        genes.append(set(tissue.columns))

    common_samples = list(set.intersection(*sample_ids))
    if 'gene_id' in common_samples:
        common_samples.remove('gene_id')

    common_genes = list(set.intersection(*genes))
    no_common_sample = len(common_samples)
    no_common_gene = len(common_genes)
    print("Kartik")
    print(common_samples)
    print(common_genes)
    print("Kartik")

    most_var_genes = set()


    print("\n[Find 2000 most varying genes for each tissue]")
    print("-------------------------------------------------")
    for i in tqdm(range(n), total=n, desc="Tissues Processed: ", unit=" Tissue(s)"):
        tissues[i] = tissues[i].loc[common_samples]
        print("GK")
        print(tissues[i])
        tissues[i] = tissues[i].loc[:, ~tissues[i].columns.duplicated()]
        tissues[i] = tissues[i][common_genes]
        tissues[i] = tissues[i].apply(pd.to_numeric, errors='coerce')


        numeric_columns = tissues[i].select_dtypes(include=np.number)
        print(numeric_columns)
        if not numeric_columns.empty:
            variances = np.var(numeric_columns, axis=0)
            sorted_indices = np.argsort(variances)[::-1][:2000]
            sorted_columns = numeric_columns.columns[sorted_indices]
            most_var_genes.update(sorted_columns)


    most_var_genes = list(most_var_genes)
    print(most_var_genes)
    gene_count = []

    for i in range(n):
        tissues[i] = tissues[i][most_var_genes]

        gene_count.append(len(tissues[i].columns))
        tissues[i].columns = [f"{gene}.{i+1}" for gene in tissues[i].columns]

    tissues_df = pd.concat(tissues, axis=1, ignore_index=False)

    print("\n[Creating correlation matrix]")
    print("-------------------------------------------------")
    with tqdm(total=len(tissues_df.columns), desc="Column-pairs Processed: ", unit=" Column-pair(s)") as pbar:
        def update_pbar(x):
            pbar.update()
            return x
        correlation_matrix = tissues_df.corr(method="spearman", min_periods=1).apply(update_pbar, axis=1)

    correlation_matrix = correlation_matrix.abs()


    tissues_df = pd.concat(tissues, axis=1, ignore_index=False)

    print("\n[Creating correlation matrix]")
    print("-------------------------------------------------")
    with tqdm(total=len(tissues_df.columns), desc="Column-pairs Processed: ", unit=" Column-pair(s)") as pbar:
        def update_pbar(x):
            pbar.update()
            return x
        correlation_matrix = tissues_df.corr(method="spearman", min_periods=1).apply(update_pbar, axis=1)

    correlation_matrix = correlation_matrix.abs()


    return correlation_matrix, gene_count

# Function to check if a file has an allowed extension
def allowed_file(filename):
    ALLOWED_EXTENSIONS = {"zip", "gz"}
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS


In [ ]:
A, gene_count = create_corr_matrix(tNames)

['MSBB']

[Reading gene expression data for each tissue]
-------------------------------------------------


Tissues Processed:   0%|          | 0/1 [00:00<?, ? Tissue(s)/s]

MSBB
/content/Kidney_Cortex.v8.normalized_expression.bed.gz


Tissues Processed: 100%|██████████| 1/1 [00:02<00:00,  2.38s/ Tissue(s)]


                GENEID         SYMBOL
0      ENSG00000227232         WASH7P
1      ENSG00000186092          OR4F5
2      ENSG00000268903  RP11-34P13.15
3      ENSG00000269981  RP11-34P13.16
4      ENSG00000241860  RP11-34P13.13
...                ...            ...
24765  ENSG00000224533      TMLHE-AS1
24766  ENSG00000185973          TMLHE
24767  ENSG00000168939          SPRY3
24768  ENSG00000124333          VAMP7
24769  ENSG00000182484         WASH6P

[24770 rows x 2 columns]
Kartik
['GTEX-1QP6S', 'GTEX-147F4', 'GTEX-1JMPZ', 'GTEX-15CHQ', 'GTEX-1399S', 'GTEX-12WSG', 'GTEX-13OVL', 'GTEX-14E6D', 'GTEX-1A3MW', 'GTEX-1C6VQ', 'GTEX-1F48J', 'GTEX-ZYT6', 'GTEX-ZC5H', 'GTEX-1E2YA', 'GTEX-1EKGG', 'GTEX-14C39', 'GTEX-13OVI', 'GTEX-1JMQK', 'GTEX-1A8FM', 'GTEX-T5JC', 'GTEX-NPJ8', 'GTEX-13OW6', 'GTEX-17F97', 'GTEX-ZYFG', 'GTEX-1AX8Z', 'GTEX-ZDXO', 'GTEX-1GF9W', 'GTEX-13112', 'GTEX-QDT8', 'GTEX-T6MN', 'GTEX-1N2DW', 'GTEX-O5YV', 'GTEX-11TTK', 'GTEX-1LG7Y', 'GTEX-1NV8Z', 'GTEX-1PWST', 'GTEX-1497J', '

...

Tissues Processed: 100%|██████████| 1/1 [00:03<00:00,  3.69s/ Tissue(s)]


SYMBOL      RP11-388M20.1     AMY2A    PRSS44  RP11-498P14.3       ACR  \
GTEX-1QP6S      -0.274391 -1.236652  0.880888       1.744448 -0.135911   
GTEX-147F4       0.135911  1.102440 -0.493553       0.832006  0.345485   
GTEX-1JMPZ      -0.832006  0.135911  0.532190       2.211127  1.494155   
GTEX-15CHQ       1.102440  0.695908  0.985610      -0.571638  1.744448   
GTEX-1399S       0.455640  0.455640 -0.785036      -1.494155 -0.418371   
...                   ...       ...       ...            ...       ...   
GTEX-QDVN        0.418371  1.166919 -0.418371       0.345485  0.239380   
GTEX-1BAJH       0.170185  1.236652  1.926403      -2.211127  0.985610   
GTEX-11OF3      -0.204660  0.381675  0.653377      -1.744448  0.033880   
GTEX-13NYB      -1.397837 -0.832006  0.695908      -0.033880  0.611996   
GTEX-1CB4J       0.985610  0.653377 -1.926403      -0.985610  0.418371   

SYMBOL        NDUFB3  ACTG1P15  RP11-470M17.2  LINC00379  AC091133.1  ...  \
GTEX-1QP6S  0.309743  0.000000    

Column-pairs Processed: 100%|██████████| 2000/2000 [00:00<00:00, 3162.69 Column-pair(s)/s]



[Creating correlation matrix]
-------------------------------------------------


Column-pairs Processed: 100%|██████████| 2000/2000 [00:00<00:00, 3347.24 Column-pair(s)/s]


In [ ]:
A

,GOT1.1,ZSCAN22.1,POT1.1,PKN3.1,PRIM2.1,TRIB2.1,FAM168B.1,LINC00623.1,IMMT.1,MRVI1.1,...,RP11-730G20.2.1,FAM46C.1,ZNF600.1,LMTK2.1,MRPL50.1,BEST1.1,FHL5.1,RP11-91J19.2.1,RP11-706O15.1.1,IQCH.1
GOT1.1,1.000000,0.186764,0.218824,0.299505,0.244871,0.217538,0.488874,0.385298,0.800148,0.473825,...,0.256287,0.056370,0.268795,0.186379,0.457746,0.093189,0.238826,0.183195,0.291852,0.095054
ZSCAN22.1,0.186764,1.000000,0.442569,0.133578,0.127886,0.254325,0.012059,0.202521,0.093382,0.295357,...,0.153129,0.124317,0.131070,0.286674,0.084732,0.217602,0.008747,0.094797,0.190012,0.355586
POT1.1,0.218824,0.442569,1.000000,0.184288,0.236253,0.000386,0.031127,0.161039,0.039617,0.241495,...,0.099331,0.092321,0.072995,0.168821,0.112580,0.180430,0.197280,0.166377,0.096437,0.344395
PKN3.1,0.299505,0.133578,0.184288,1.000000,0.124928,0.401183,0.361792,0.180397,0.315808,0.109171,...,0.370538,0.231237,0.446427,0.144254,0.254936,0.171683,0.302592,0.202682,0.174706,0.058010
PRIM2.1,0.244871,0.127886,0.236253,0.124928,1.000000,0.064667,0.267895,0.037848,0.079040,0.450479,...,0.326870,0.037044,0.097981,0.018490,0.302688,0.063702,0.441829,0.077240,0.039134,0.021834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BEST1.1,0.093189,0.217602,0.180430,0.171683,0.063702,0.253650,0.308991,0.238408,0.133192,0.172583,...,0.208406,0.118400,0.124703,0.506624,0.048910,1.000000,0.352273,0.110779,0.223905,0.531320
FHL5.1,0.238826,0.008747,0.197280,0.302592,0.441829,0.432182,0.297125,0.168982,0.027236,0.383047,...,0.169593,0.228696,0.136407,0.094283,0.086051,0.352273,1.000000,0.078622,0.211718,0.021738
RP11-91J19.2.1,0.183195,0.094797,0.166377,0.202682,0.077240,0.156377,0.194353,0.257766,0.215126,0.037687,...,0.103704,0.232845,0.222683,0.129848,0.099717,0.110779,0.078622,1.000000,0.186379,0.013634
RP11-706O15.1.1,0.291852,0.190012,0.096437,0.174706,0.039134,0.326259,0.312239,0.138369,0.254164,0.076243,...,0.163805,0.364654,0.457972,0.108978,0.263843,0.223905,0.211718,0.186379,1.000000,0.105602


In [ ]:
tissues

{'Name': ['MSBB'],
 'exp': ['/content/Kidney_Cortex.v8.normalized_expression.bed.gz']}

In [ ]:
gene_count

[2000]

In [ ]:
n = len(A)/len(tissues)

In [ ]:
n

1000.0

In [ ]:
import time
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import copy


def unit_vector(vector):
    """Returns the unit vector of the vector."""
    return vector / np.linalg.norm(vector)


def angle_between(v1, v2):
    """Returns the angle in radians between vectors 'v1' and 'v2'::

    >>> angle_between((1, 0, 0), (0, 1, 0))
    1.5707963267948966
    >>> angle_between((1, 0, 0), (1, 0, 0))
    0.0
    >>> angle_between((1, 0, 0), (-1, 0, 0))
    3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


# MultiCens: local centrality
def local_centrality(A_tilde_full, num_layers, p):
    print("\n[Computing Local Centrality]\n")
    print("Im in")
    print(A_tilde_full)
    print("im something")
    print(num_layers)
    print("By")
    print(p)
    start_time = time.time()
    A_tilde_full = A_tilde_full / np.sum(A_tilde_full, axis=0)
    # A_tilde_full = A_tilde_full/LA.norm(A_tilde_full)
    n = int(np.shape(A_tilde_full)[0] / num_layers)
    N = int(np.shape(A_tilde_full)[0])
    A_tilde = np.zeros_like(A_tilde_full, dtype=np.float32)
    print("middle")
    for i in range(num_layers):
        A_tilde[(i * n) : ((i + 1) * n), (i * n) : ((i + 1) * n)] = A_tilde_full[
            (i * n) : ((i + 1) * n), (i * n) : ((i + 1) * n)
        ]

    ones_t = np.ones((N,)) / N
    l = np.copy(ones_t)
    l_new = np.copy(ones_t)



    count = 0
    current_angle = np.zeros(
        3,
    )
    # print("Local Centrality computation starts")
    print("somewhere")
    while count < 200:
        # print(count)
        count = count + 1
        l_new = (p * ((A_tilde).dot(l))) + (1 - p) * ones_t
        current_angle[0] = current_angle[1]
        current_angle[1] = current_angle[2]
        current_angle[2] = angle_between(l, l_new)
        # print(current_angle)
        if ((current_angle[1] == current_angle[2])) or (current_angle[2] == 0):
            break
        l = copy.deepcopy(l_new)
        # l = copy.deepcopy(l_new)/LA.norm(l_new)
    # print("l values")
    # print(l)
    # new_l = l/l.sum()# + l_tt/l_tt.sum()
    new_l = copy.deepcopy(l)
    # ---new_l is empty
    for i in range(num_layers):
        l[(i * n) : ((i + 1) * n)] = (
            l[(i * n) : ((i + 1) * n)] / l[(i * n) : ((i + 1) * n)].sum()
        )

    new_l = new_l / np.max(new_l)
    print("end")
    # Time calculation
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = round(execution_time/60, 2)
    time_taken = f'{execution_time} min(s)'
    print("\nTime taken to compute the local centrality scores:", time_taken, "\n")
    return new_l


# MultiCens: global centrality
def global_centrality(A_tilde_full, num_layers, p):
    print("\n[Computing Global Centrality]\n")
    start_time = time.time()
    l = local_centrality(A_tilde_full, num_layers, p)  # last checkpoint
    # A_tilde = copy.deepcopy(A_tilde_full)
    A_tilde = A_tilde_full / np.sum(A_tilde_full, axis=0)
    N = int(np.shape(A_tilde)[0])
    n = int(N / num_layers)
    # print("A_tilde sums")
    # print(np.sum(A_tilde[0,:]))
    # print(np.sum(A_tilde[:,0]))

    A = np.zeros_like(A_tilde_full, dtype=np.float32)
    C = np.zeros_like(A_tilde_full, dtype=np.float32)
    A[:n, :n] = copy.deepcopy(A_tilde[:n, :n])
    A[n : (2 * n), n : (2 * n)] = copy.deepcopy(A_tilde[n : (2 * n), n : (2 * n)])
    A[(2 * n) : (3 * n), (2 * n) : (3 * n)] = copy.deepcopy(
        A_tilde[(2 * n) : (3 * n), (2 * n) : (3 * n)]
    )
    A[(3 * n) : (4 * n), (3 * n) : (4 * n)] = copy.deepcopy(
        A_tilde[(3 * n) : (4 * n), (3 * n) : (4 * n)]
    )
    C = A_tilde - A

    ones_t = np.ones((N,)) / N
    g = copy.deepcopy(ones_t)
    g_new = copy.deepcopy(ones_t)

    counter = 0
    current_angle = np.zeros(
        3,
    )

    while counter < 150:
        g_new = (p * ((A + C).dot(g) + (C.dot(l)))) + ((1 - p) * ones_t)

        current_angle[0] = current_angle[1]
        current_angle[1] = current_angle[2]
        current_angle[2] = angle_between(g, g_new)
        # print(current_angle)
        if ((current_angle[1] == current_angle[2])) or (current_angle[2] == 0):
            break
        g = copy.deepcopy(g_new)
        # g = copy.deepcopy(g_new)/LA.norm(g_new)
        counter += 1
        # print(counter)
        # print(np.sum(g))

    new_g = copy.deepcopy(g)
    g_fresh = copy.deepcopy(g)
    new_g = new_g / new_g.sum()
    g_fresh = g_fresh / g_fresh.sum()

    new_g = new_g / np.max(new_g)

    # Time calculation
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = round(execution_time/60, 2)
    time_taken = f'{execution_time} min(s)'
    print("\nTime taken to compute the global centrality scores:", time_taken, "\n")

    return new_g


def right_new_local_centrality_st(
    A_tilde_full, num_layers, target_tissue, target_gene_indices, start, end, p
):
    # print("target tissue id")
    # print(target_tissue)
    # print("right_new_local_centrality")
    A_tilde_full = A_tilde_full / np.sum(A_tilde_full, axis=0)
    # A_tilde_full = A_tilde_full/LA.norm(A_tilde_full)
    num_target_genes = int(np.shape(target_gene_indices)[0])
    n = int(np.shape(A_tilde_full)[0] / num_layers)
    N = int(np.shape(A_tilde_full)[0])
    A_tilde = np.zeros_like(A_tilde_full, dtype=np.float32)

    # if target_tissue == 1:
    #     A_tilde[n:,n:] = A_tilde_full[n:,n:]
    # elif target_tissue == 0:
    #     A_tilde[:n,:n] = A_tilde_full[:n,:n]
    # else:
    #     print("wrong target tissue")

    A_tilde[start:end, start:end] = A_tilde_full[
        start:end, start:end
    ]  # replacement for above commented code

    np.fill_diagonal(A_tilde, 0.0)
    # A_tilde = A_tilde/LA.norm(A_tilde)

    ones_t = np.zeros((N,))
    # ones_t[
    #     (np.asarray(target_gene_indices, dtype=np.int32) + int(n * target_tissue))
    # ] = 1 / len(target_gene_indices)

    ones_t[np.asarray(target_gene_indices, dtype=np.int32)] = 1 / len(
        target_gene_indices
    )  # replacement for above commented code

    l = np.copy(ones_t)
    l_new = np.copy(ones_t)

    count = 0
    current_angle = np.zeros(
        3,
    )
    print("[Local Centrality computation starts]")
    while count < 200:
        # print(count)
        count = count + 1
        l_new = (p * ((A_tilde).dot(l))) + (1 - p) * ones_t
        current_angle[0] = current_angle[1]
        current_angle[1] = current_angle[2]
        current_angle[2] = angle_between(l, l_new)
        # print(current_angle)
        if (
            (current_angle[0] == current_angle[1])
            and (current_angle[1] == current_angle[2])
        ) or (current_angle[2] == 0):
            break
        l = copy.deepcopy(l_new)
        # l = copy.deepcopy(l_new)/LA.norm(l_new)
    # print("l values")
    # print(l)
    # new_l = l/l.sum()# + l_tt/l_tt.sum()
    new_l = copy.deepcopy(l)
    # if target_tissue == 0:
    #     print("Found local centrality for target set centrality")
    #     new_l[:n] = l[:n] / l[:n].sum()
    # elif target_tissue == 1:
    #     new_l[n:] = l[n:] / l[n:].sum()
    #     print("Found local centrality for source set centrality")
    # else:
    #     print("invalid target tissue")

    new_l[start:end] = (
        l[start:end] / l[start:end].sum()
    )  # replacement for above commented code

    # print("Target genes")
    # print(target_gene_indices)
    return new_l


def right_target_global_centrality_t(
    A_tilde_full, num_layers, target_tissue, target_gene_indices, start, end, p
):
    print("\n[Computing Query-set Centrality]\n")
    start_time = time.time()
    l = right_new_local_centrality_st(
        A_tilde_full, num_layers, target_tissue, target_gene_indices, start, end, p
    )  # last checkpoint
    # A_tilde = copy.deepcopy(A_tilde_full)
    A_tilde = A_tilde_full / np.sum(A_tilde_full, axis=0)
    N = int(np.shape(A_tilde)[0]) # rows in correlation matrix
    n = int(N / num_layers) # number of genes selected from each tissue
    # print("A_tilde sums")
    # print(np.sum(A_tilde[0, :]))
    # print(np.sum(A_tilde[:, 0]))

    A = np.zeros_like(A_tilde_full, dtype=np.float32)
    C = np.zeros_like(A_tilde_full, dtype=np.float32)
    # A[:n, :n] = copy.deepcopy(A_tilde[:n, :n])
    # A[n:, n:] = copy.deepcopy(A_tilde[n:, n:])
    # C[:n, n:] = copy.deepcopy(A_tilde[:n, n:])
    # C[n:, :n] = copy.deepcopy(A_tilde[n:, :n])

    # replacement for above commented code
    # N is the number of sub-matrices
    # n size of each sub-matrix
    for i in range(num_layers):
        for j in range(num_layers):
            if i == j:
                A[i*n:(i+1)*n, j*n:(j+1)*n] = copy.deepcopy(A_tilde[i*n:(i+1)*n, j*n:(j+1)*n])
            else:
                C[i*n:(i+1)*n, j*n:(j+1)*n] = copy.deepcopy(A_tilde[i*n:(i+1)*n, j*n:(j+1)*n])

    print(f"[Finding target global centrality for layer {str(target_tissue+1)}]")

    ones_t = np.zeros((N,))
    # ones_t[
    #     (np.asarray(target_gene_indices, dtype=np.int32) + int(n * target_tissue))
    # ] = 1 / len(target_gene_indices)

    ones_t[np.asarray(target_gene_indices, dtype=np.int32)] = 1 / len(
        target_gene_indices
    )  # replacement for above commented code
    g = copy.deepcopy(ones_t)
    g_new = copy.deepcopy(ones_t)

    counter = 0
    current_angle = np.zeros(
        3,
    )
    while counter < 200:
        g_new = (p * ((A + C).dot(g) + (C.dot(l)))) + ((1 - p) * ones_t)

        current_angle[0] = current_angle[1]
        current_angle[1] = current_angle[2]
        current_angle[2] = angle_between(g, g_new)
        # print(current_angle)
        if (
            (current_angle[0] == current_angle[1])
            and (current_angle[1] == current_angle[2])
        ) or (current_angle[2] == 0):
            break
        g = copy.deepcopy(g_new)
        # g = copy.deepcopy(g_new)/LA.norm(g_new)
        counter += 1
        # print(counter)
        # print(np.sum(g))
    new_g = copy.deepcopy(g)
    # if target_tissue == 1:
    #     print("Found target set centrality")
    #     new_g[:n] = g[:n] / g[:n].max()
    # elif target_tissue == 0:
    #     new_g[n:] = g[n:] / g[n:].max()
    #     print("Found source set centrality")
    # else:
    #     print("invalid target tissue")

    new_g[start:end] = (
        g[start:end] / g[start:end].max()
    )  # replacement for above commented code

     # Time calculation
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = round(execution_time/60, 2)
    time_taken = f'{execution_time} min(s)'
    print("\nTime taken to compute the query-set centrality scores:", time_taken, "\n")

    return l, new_g


def plot_k_curve(genes, cen_vector, ground_truth_genes, filtered, n):
    secreted_proteins = list(
        pd.read_csv("../data/002790_proteins.csv", header=None).values
    )
    secreted_proteins = [s[0].upper() for s in secreted_proteins]
    order = cen_vector[:n].argsort()
    ranks = order.argsort()
    d = {"gene_name": genes, "centrality": cen_vector[:n], "rank": n - ranks}
    results = pd.DataFrame(data=d)
    results = results.sort_values(by=["centrality"], ascending=False)
    ranked_genes = results["gene_name"].tolist()

    filtered_results = results[results.gene_name.isin(secreted_proteins)]
    filtered_ranked_genes = filtered_results["gene_name"].tolist()

    if filtered:
        n_filtered = filtered_results.shape[0]
        k_range = np.arange(n_filtered)
        hits = np.zeros_like(k_range)
        current_hit_count = 0
        for i in k_range:
            if filtered_ranked_genes[i] in ground_truth_genes:
                current_hit_count = current_hit_count + 1
            hits[i] = current_hit_count
        random_curve = np.cumsum(np.full((n_filtered,), hits[-1] / n_filtered))
        plot_variables = {"recall_at_k": hits, "random_curve": random_curve}
        plot_df = pd.DataFrame(data=plot_variables)
        sns.lineplot(data=plot_df)

    else:
        k_range = np.arange(n)
        hits = np.zeros_like(k_range)
        current_hit_count = 0
        for i in k_range:
            if ranked_genes[i] in ground_truth_genes:
                current_hit_count = current_hit_count + 1
            hits[i] = current_hit_count
        random_curve = np.cumsum(np.full((n,), hits[-1] / n))
        plot_variables = {"query-set centrality": hits, "random_curve": random_curve}
        plot_df = pd.DataFrame(data=plot_variables)
        sns.lineplot(data=plot_df)
        # plt.xlabel("Top k predictions", labelpad=1)
        # plt.ylabel("Recall at k", labelpad=1)
        plt.xticks(np.arange(0, 15001, 3500))
        plt.savefig(
            "./insulin_responding_results_corr_SNAP.svg",
            dpi=300,
            fontsize=17,
            bbox_inches="tight",
        )
        # plt.title("My Daily Step Count Tracked by Fitbit", y=1.02, fontsize=22);
        max_area = (current_hit_count * (current_hit_count + 1)) / 2 + (
            (n - current_hit_count) * current_hit_count
        )
        method_area = np.sum(hits)
        print("Area under curve is: ", (method_area / max_area))

    # get lncRNAs ranking
    lncRNAs = list(np.load("../data/paper/lncRNAs.npy"))
    lncRNA_results = results[results.gene_name.isin(lncRNAs)]
    lncRNA_ranked_genes = lncRNA_results["gene_name"].tolist()

    print(hits[-1])
    return (plot_df, results, filtered_results, lncRNA_results)


In [ ]:
result = pd.DataFrame()


NameError: name 'pd' is not defined

In [ ]:
 if measure == 'query':
    query_tissue = input('query-tissue')
    print("Upload your query file:")
    query_file = files.upload()

    # Ensure query_index is properly retrieved and converted to an integer
    try:
        query_index = tissue.index(query_tissue)
        print(f"The index of '{query_tissue}' in the list is: {query_index}")
    except ValueError:
        print(f"'{query_tissue}' is not found in the list of tissues.")

    s = int(query_index * n)
    e = int(s + n)
    query_gene_list = pd.read_csv(query_file, header=None)[0].tolist()
    query_gene_list = [f"{gene}.{query_index+1}" for gene in query_gene_list]
    genes = A.columns[s:e]
    common_target_genes = np.intersect1d(genes, query_gene_list)
    genes_indices = [i for i, e in enumerate(A.columns) if e in common_target_genes]
    _, g = right_target_global_centrality_t(A.values, num_layers=len(tissues), target_tissue=query_index, target_gene_indices=genes_indices, start=s, end=e, p=0.9)
    result['Centrality'] = g
elif measure == 'global':
    result['Centrality'] = global_centrality(A.values, len(tissues), p=0.9)
else:
    result['Centrality'] = local_centrality(A.values, len(tissues), p=0.9)


[Computing Local Centrality]

Im in
[[1.         0.18676442 0.21882436 ... 0.18319506 0.29185157 0.09505434]
 [0.18676442 1.         0.44256865 ... 0.09479709 0.19001222 0.35558557]
 [0.21882436 0.44256865 1.         ... 0.16637726 0.09643707 0.34439514]
 ...
 [0.18319506 0.09479709 0.16637726 ... 1.         0.18637855 0.01363432]
 [0.29185157 0.19001222 0.09643707 ... 0.18637855 1.         0.10560165]
 [0.09505434 0.35558557 0.34439514 ... 0.01363432 0.10560165 1.        ]]
im something
2
By
0.9
middle
somewhere
end

Time taken to compute the local centrality scores: 0.0 min(s) 



In [ ]:
tissues['Name'][0]

'MSBB'

In [ ]:
# Get the tissue name
tissue_name = tissues['Name'][0]

# Generate name_list for the tissue
name_list = [tissue_name for _ in range(gene_count[0])]

In [ ]:
name_list

['MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 

In [ ]:
#name_list = [[tissues['Name'][i] for _ in range(gene_count[i])] for i, tissue in enumerate(tissues)]


In [ ]:


result['Tissue Name'] =  name_list

In [ ]:

result['Gene Name'] = [re.sub(r'\.\d+$', '', s) for s in A.columns]

In [ ]:
result['Tissue Name']

['MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 'MSBB',
 

In [ ]:
def process_result(result, measure, tissues):
    if measure == 'local':
        ranking = pd.DataFrame()
        for tissue_name in tissues['Name']:
            df = result[result['Tissue Name'] == tissue_name]
            df = df.sort_values(by='Centrality', ascending=False)
            df['Rank'] = range(1, len(df) + 1)
            ranking = pd.concat([ranking, df], ignore_index=True)
        result = ranking
    else:
        if isinstance(result, pd.DataFrame):
            result = result.sort_values(by='Centrality', ascending=False)
            result['Rank'] = range(1, len(result) + 1)
        elif isinstance(result, list) and all(isinstance(entry, dict) for entry in result):
            # Sort the list of dictionaries by 'Centrality'
            result = sorted(result, key=lambda x: x['Centrality'], reverse=True)
            # Add 'Rank' to each dictionary entry
            for i, entry in enumerate(result):
                entry['Rank'] = i + 1
        else:
            return jsonify({"error": "Invalid result format. Expected DataFrame or list of dictionaries."}), 400

    result = result[['Tissue Name', 'Gene Name', 'Centrality', 'Rank']]
    return result


In [ ]:
# Count the total number of keys in the dictionary
total_keys = len(result.keys())

# Count the total number of values in the dictionary
total_values = len(result.values())

# Output the total number of keys and values
print("Total keys in the dictionary:", total_keys)
print("Total values in the dictionary:", total_values)


Total keys in the dictionary: 3
Total values in the dictionary: 3


In [ ]:
result['Centrality'] = np.round(result['Centrality'], 4)

In [ ]:
result

{'Centrality': array([0.9099, 0.7792, 0.8059, ..., 0.6833, 0.7595, 0.8822]),
 'Tissue Name': ['MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB',
  'MSBB'

In [ ]:
df_result = pd.DataFrame.from_dict(result)

In [ ]:
df_result

,Centrality,Tissue Name,Gene Name
0,0.9099,MSBB,GOT1
1,0.7792,MSBB,ZSCAN22
2,0.8059,MSBB,POT1
3,0.8538,MSBB,PKN3
4,0.8487,MSBB,PRIM2
...,...,...,...
1995,0.9476,MSBB,BEST1
1996,0.8163,MSBB,FHL5
1997,0.6833,MSBB,RP11-91J19.2
1998,0.7595,MSBB,RP11-706O15.1


In [ ]:
# Assuming df_result_sorted is your sorted DataFrame
df_result_sorted = df_result.sort_values(by='Centrality', ascending=False).reset_index(drop=True)

# Output the modified DataFrame
print(df_result_sorted)


      Centrality Tissue Name      Gene Name
0         1.0000        MSBB          NDST2
1         0.9995        MSBB          GHITM
2         0.9990        MSBB        ALDH5A1
3         0.9973        MSBB          MOCS2
4         0.9938        MSBB           TEP1
...          ...         ...            ...
1995      0.6228        MSBB          PIPSL
1996      0.6217        MSBB    AP003068.23
1997      0.6150        MSBB  RP11-488L18.6
1998      0.6098        MSBB        RPL24P4
1999      0.5925        MSBB       HLA-DRB5

[2000 rows x 3 columns]
